## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re
from collections import *
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
cnt = Counter()
for sentence in sentences_ted:
    for word in sentence:
        cnt[word] += 1
        
counts_ted_top1000 = []
for common_word in cnt.most_common(1000):
    counts_ted_top1000.append(common_word[1])

Plot distribution of top-1000 words

In [14]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [15]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [16]:
model_ted.most_similar("man")

[('woman', 0.8359224200248718),
 ('guy', 0.8224167227745056),
 ('lady', 0.7852599024772644),
 ('boy', 0.7541414499282837),
 ('girl', 0.7390031814575195),
 ('gentleman', 0.7352846264839172),
 ('soldier', 0.7309067845344543),
 ('kid', 0.698061466217041),
 ('poet', 0.6628059148788452),
 ('john', 0.6622768044471741)]

In [17]:
model_ted.most_similar("woman")

[('girl', 0.8370451331138611),
 ('man', 0.8359224200248718),
 ('lady', 0.7914308905601501),
 ('boy', 0.7843156456947327),
 ('child', 0.7535058259963989),
 ('soldier', 0.7467571496963501),
 ('nurse', 0.728182315826416),
 ('kid', 0.7145192623138428),
 ('gentleman', 0.7079211473464966),
 ('doctor', 0.6912469863891602)]

In [18]:
model_ted.most_similar("dream")

[('legacy', 0.6925253868103027),
 ('client', 0.6840232014656067),
 ('passion', 0.6326696872711182),
 ('son', 0.6255491375923157),
 ('faith', 0.6254357695579529),
 ('profession', 0.621522068977356),
 ('grandmother', 0.6143438220024109),
 ('father', 0.597388744354248),
 ('promise', 0.5972912311553955),
 ('daughter', 0.5854037404060364)]

In [19]:
model_ted.most_similar("marriage")

[('equality', 0.6786928772926331),
 ('illness', 0.67664635181427),
 ('depression', 0.649956226348877),
 ('likelihood', 0.6346561312675476),
 ('condition', 0.6305301189422607),
 ('faith', 0.6272737979888916),
 ('religion', 0.6228148937225342),
 ('penalty', 0.6192631721496582),
 ('customer', 0.6182265281677246),
 ('status', 0.6177093386650085)]

In [20]:
model_ted.most_similar("technology")

[('tool', 0.704346239566803),
 ('design', 0.6995739340782166),
 ('technologies', 0.6903078556060791),
 ('knowledge', 0.6835100054740906),
 ('innovation', 0.6754068732261658),
 ('software', 0.6695283651351929),
 ('method', 0.6621416211128235),
 ('tools', 0.6563212275505066),
 ('digital', 0.649803638458252),
 ('biology', 0.6477077007293701)]

In [21]:
model_ted.most_similar("entertainment")

[('sciences', 0.7525447607040405),
 ('engineering', 0.748826265335083),
 ('educational', 0.7400134801864624),
 ('humanities', 0.7197060585021973),
 ('computing', 0.715947151184082),
 ('communications', 0.7136222124099731),
 ('exploration', 0.7092671990394592),
 ('strategic', 0.7015290260314941),
 ('organizing', 0.6889089941978455),
 ('telecommunications', 0.683088481426239)]

In [22]:
model_ted.most_similar("design")

[('technology', 0.6995739340782166),
 ('biology', 0.696924090385437),
 ('engineering', 0.6926122903823853),
 ('model', 0.6853285431861877),
 ('mathematics', 0.6803268790245056),
 ('mathematical', 0.6735076308250427),
 ('science', 0.6733931303024292),
 ('architecture', 0.672319233417511),
 ('tool', 0.6662320494651794),
 ('intelligence', 0.6210753321647644)]

In [23]:
print(model_ted.most_similar_cosmul(positive=['woman', 'king'], negative=['man']))
print(model_ted.most_similar_cosmul(positive=['computer', 'mind'], negative=['man']))
print(model_ted.most_similar_cosmul(positive=['life','meaning']))
print(model_ted.most_similar_cosmul(positive=['war', 'usa'])[:3])

[('queen', 0.9333152174949646), ('mary', 0.9283277988433838), ('president', 0.9175340533256531), ('luther', 0.9161136150360107), ('named', 0.9057334661483765), ('james', 0.9025154113769531), ('edward', 0.9017987847328186), ('robert', 0.8967223763465881), ('german', 0.8920257687568665), ('french', 0.8916367292404175)]
[('memory', 1.318468451499939), ('biology', 1.2794066667556763), ('3d', 1.2191344499588013), ('physical', 1.2084577083587646), ('testing', 1.20833158493042), ('knowledge', 1.1944022178649902), ('intelligence', 1.1899983882904053), ('biological', 1.1796506643295288), ('visual', 1.1779025793075562), ('brain', 1.1743439435958862)]
[('experience', 0.6385869979858398), ('happiness', 0.6376583576202393), ('consciousness', 0.6104937195777893), ('joy', 0.6081647872924805), ('humanity', 0.6064414978027344), ('sense', 0.6018146276473999), ('desire', 0.5992480516433716), ('memory', 0.5989620089530945), ('existence', 0.5951810479164124), ('purpose', 0.5949209928512573)]
[('soviet', 0.

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [24]:
words_top_ted = []
for common_word in cnt.most_common(1000):
    words_top_ted.append(common_word[0])

# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [25]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [26]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Extra: Literatura

In [27]:
with open('/home/matheus/Documents/data/copilados', 'r') as in_f:
    liter_text = in_f.read().replace('!','.').replace('?','.').lower().split('.')

liter_text[:3]

['\x0c1\n\nnao se sabe se kublai khan acredita em tudo o que diz marco polo quando este lhe\ndescreve as cidades visitadas em suas missoes diplomaticas, mas o imperador dos\ntartaros certamente continua a ouvir o jovem veneziano com maior curiosidade e\natencao do que a qualquer outro de seus enviados ou exploradores',
 ' existe um momento\nna vida dos imperadores que se segue ao orgulho pela imensa amplitude dos territorios\nque conquistamos, a melancolia e ao alivio de saber que em breve desistiremos de\nconhece-los e compreende-los, uma sensacao de vazio que surge ao calar da noite com\no odor dos elefantes apos a chuva e das cinzas de sandalo que se resfriam nos braseiros,\numa vertigem que faz estremecer os rios e as montanhas historiadas nos fulvos dorsos\ndos planisferios, enrolando um depois do outro os despachos que anunciam o\naniquilamento dos ultimos exercitos inimigos de derrota em derrota, e abrindo o lacre\ndos sinetes de reis dos quais nunca se ouviu falar e que implora

In [28]:
for s_i in range(len(liter_text)):
    liter_text[s_i] = re.sub("[^a-z]", " ", liter_text[s_i]).split()

liter_text[:2]

[['nao',
  'se',
  'sabe',
  'se',
  'kublai',
  'khan',
  'acredita',
  'em',
  'tudo',
  'o',
  'que',
  'diz',
  'marco',
  'polo',
  'quando',
  'este',
  'lhe',
  'descreve',
  'as',
  'cidades',
  'visitadas',
  'em',
  'suas',
  'missoes',
  'diplomaticas',
  'mas',
  'o',
  'imperador',
  'dos',
  'tartaros',
  'certamente',
  'continua',
  'a',
  'ouvir',
  'o',
  'jovem',
  'veneziano',
  'com',
  'maior',
  'curiosidade',
  'e',
  'atencao',
  'do',
  'que',
  'a',
  'qualquer',
  'outro',
  'de',
  'seus',
  'enviados',
  'ou',
  'exploradores'],
 ['existe',
  'um',
  'momento',
  'na',
  'vida',
  'dos',
  'imperadores',
  'que',
  'se',
  'segue',
  'ao',
  'orgulho',
  'pela',
  'imensa',
  'amplitude',
  'dos',
  'territorios',
  'que',
  'conquistamos',
  'a',
  'melancolia',
  'e',
  'ao',
  'alivio',
  'de',
  'saber',
  'que',
  'em',
  'breve',
  'desistiremos',
  'de',
  'conhece',
  'los',
  'e',
  'compreende',
  'los',
  'uma',
  'sensacao',
  'de',
  'vazio',


In [29]:
shuffle(liter_text)
print(len(liter_text))

296415


In [30]:
model_liter = Word2Vec(liter_text, min_count=5)

In [31]:
model_liter.most_similar("homem")

[('animal', 0.754645824432373),
 ('rapaz', 0.7056355476379395),
 ('ladrao', 0.6170637011528015),
 ('individuo', 0.6123188734054565),
 ('pintor', 0.6094642877578735),
 ('fantasma', 0.6024704575538635),
 ('imbecil', 0.6024476885795593),
 ('idiota', 0.6012296080589294),
 ('sujeito', 0.5954297780990601),
 ('menino', 0.5878980159759521)]

In [32]:
model_liter.most_similar("mulher")

[('moca', 0.759548008441925),
 ('crianca', 0.7408807873725891),
 ('menina', 0.7214410305023193),
 ('filha', 0.6986643075942993),
 ('rapariga', 0.665507972240448),
 ('mae', 0.6556188464164734),
 ('criada', 0.6460382342338562),
 ('esposa', 0.644051730632782),
 ('pessoa', 0.6338177919387817),
 ('irma', 0.6279371380805969)]

In [33]:
model_liter.most_similar("crime")

[('pecado', 0.4885657727718353),
 ('erro', 0.48810288310050964),
 ('castigo', 0.4646143317222595),
 ('absurdo', 0.46187689900398254),
 ('problema', 0.45942601561546326),
 ('disparate', 0.4564584195613861),
 ('individuo', 0.4551647901535034),
 ('fenomeno', 0.45493271946907043),
 ('sancao', 0.44988563656806946),
 ('pleito', 0.44754618406295776)]

In [34]:
model_liter.most_similar("sonho")

[('pensamento', 0.7407381534576416),
 ('delirio', 0.7086445093154907),
 ('ato', 0.6971638798713684),
 ('sono', 0.6870999932289124),
 ('casamento', 0.6839599013328552),
 ('destino', 0.6808556914329529),
 ('pesadelo', 0.6767008900642395),
 ('presente', 0.670269787311554),
 ('sentido', 0.6659073233604431),
 ('periodo', 0.6622503995895386)]

In [35]:
model_liter.most_similar("amor")

[('odio', 0.7591749429702759),
 ('temor', 0.7312495708465576),
 ('sofrimento', 0.7309755682945251),
 ('orgulho', 0.7150030136108398),
 ('coracao', 0.714794933795929),
 ('sentimento', 0.7056339979171753),
 ('destino', 0.6970595717430115),
 ('espirito', 0.6963846683502197),
 ('ciume', 0.6785852909088135),
 ('desejo', 0.6772885322570801)]

In [36]:
model_liter.most_similar("morte")

[('cegueira', 0.6672433018684387),
 ('vida', 0.6591309309005737),
 ('doenca', 0.6564456224441528),
 ('existencia', 0.6374093890190125),
 ('culpa', 0.6369156241416931),
 ('desgraca', 0.6241648197174072),
 ('velhice', 0.621222734451294),
 ('salvacao', 0.6192787885665894),
 ('noticia', 0.6157522797584534),
 ('loucura', 0.6151165962219238)]

In [37]:
print(model_liter.most_similar_cosmul(positive=['roma', 'franca'], negative=['italia']), '\n')
print(model_liter.most_similar_cosmul(positive=['barco'], negative=['navegar']), '\n')
print(model_liter.most_similar_cosmul(positive=['guerra', 'paz']), '\n')
print(model_liter.most_similar_cosmul(positive=['homem'], negative=['deus']), '\n')
print(model_liter.most_similar_cosmul(positive=['labirinto', 'xadrez']), '\n')

[('napoleao', 0.9915168285369873), ('julho', 0.9496642351150513), ('paris', 0.9333292841911316), ('luis', 0.922232449054718), ('waterloo', 0.9205167293548584), ('angouleme', 0.9121435880661011), ('marco', 0.9099788069725037), ('marques', 0.8990285992622375), ('espanha', 0.8983132839202881), ('guerra', 0.8972076177597046)] 

[('quarto', 1.5034515857696533), ('jeito', 1.494570255279541), ('carro', 1.487213134765625), ('cego', 1.4491883516311646), ('comenos', 1.4336888790130615), ('menino', 1.425858497619629), ('sitio', 1.4253324270248413), ('rapaz', 1.423872709274292), ('hotel', 1.419245958328247), ('burro', 1.418635606765747)] 

[('espanha', 0.6176315546035767), ('familia', 0.6147134900093079), ('inglaterra', 0.6140139102935791), ('execucao', 0.6139072179794312), ('gloria', 0.6132325530052185), ('morte', 0.6042563915252686), ('ordem', 0.6017161011695862), ('humanidade', 0.5998936891555786), ('revolucao', 0.5978935956954956), ('salvacao', 0.5976195931434631)] 

[('usava', 1.7242372035980

In [38]:
cnt = Counter()
for sentence in liter_text:
    for word in sentence:
        cnt[word] += 1
        
counts_liter_top1000 = []
for common_word in cnt.most_common(1000):
    counts_liter_top1000.append(common_word[0])
    
words_top_vec_liter = model_liter[counts_liter_top1000]

In [39]:
tsne = TSNE(n_components=2, random_state=0)
words_top_liter_tsne = tsne.fit_transform(words_top_vec_liter)

In [40]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_liter_tsne[:,0],
                                    x2=words_top_liter_tsne[:,1],
                                    names=counts_liter_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [41]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [42]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [43]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)

for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split()
    
del input_text 

In [44]:
shuffle(sentences_wiki)
print(len(sentences_wiki))

# sample 1/5 of the data
# sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
# print(len(sentences_wiki))

4267112


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [45]:
cnt = Counter()
for sentence in sentences_wiki:
    for word in sentence:
        cnt[word] += 1
        
counts_wiki_top1000 = []
for common_word in cnt.most_common(1000):
    counts_wiki_top1000.append(common_word[1])

In [46]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [47]:
model_wiki = Word2Vec(sentences_wiki, min_count=10)

In [48]:
model_wiki.most_similar("man")

[('woman', 0.7665807008743286),
 ('person', 0.6645464301109314),
 ('girl', 0.6578167080879211),
 ('boy', 0.605709433555603),
 ('dog', 0.5946558117866516),
 ('soldier', 0.5913508534431458),
 ('creature', 0.5912767648696899),
 ('gambler', 0.5849231481552124),
 ('lad', 0.5789147019386292),
 ('someone', 0.5649397969245911)]

In [49]:
model_wiki.most_similar("woman")

[('girl', 0.7951010465621948),
 ('man', 0.7665807008743286),
 ('child', 0.7453534603118896),
 ('prostitute', 0.7446708083152771),
 ('person', 0.7084254026412964),
 ('teenager', 0.6845573782920837),
 ('lover', 0.6782108545303345),
 ('heroine', 0.6554056406021118),
 ('pimp', 0.6544957160949707),
 ('maid', 0.6533594727516174)]

In [50]:
model_wiki.most_similar("marriage")

[('divorce', 0.7652644515037537),
 ('betrothal', 0.7138632535934448),
 ('marriages', 0.7124296426773071),
 ('remarriage', 0.6919487714767456),
 ('marrying', 0.6844138503074646),
 ('husband', 0.6646238565444946),
 ('wife', 0.6621097922325134),
 ('daughter', 0.6324413418769836),
 ('relationship', 0.6309663653373718),
 ('widowhood', 0.6275832653045654)]

In [51]:
model_wiki.most_similar("love")

[('dreams', 0.6716759204864502),
 ('kiss', 0.6609022617340088),
 ('duet', 0.6567891240119934),
 ('romance', 0.6403593420982361),
 ('smile', 0.6326988935470581),
 ('dream', 0.6147112250328064),
 ('madly', 0.6114822626113892),
 ('passion', 0.6114453673362732),
 ('yearning', 0.6086317300796509),
 ('infatuation', 0.6067837476730347)]

In [52]:
print(model_wiki.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])[:3])
print(model_wiki.most_similar_cosmul(positive=['computer', 'mind'], negative=['machine'])[:3])
print(model_wiki.most_similar_cosmul(positive=['life','meaning'])[:3])
print(model_wiki.most_similar_cosmul(positive=['paris', 'japan'], negative=['france'])[:3])

[('empress', 0.97281414270401), ('queen', 0.9467155337333679), ('monarch', 0.9384176731109619)]
[('personality', 1.0502445697784424), ('backstory', 0.9806424379348755), ('life', 0.9755017161369324)]
[('essence', 0.5189809203147888), ('manhood', 0.5151405930519104), ('parable', 0.5136235952377319)]
[('tokyo', 0.966160774230957), ('osaka', 0.9030835032463074), ('shibuya', 0.8948038220405579)]


In [53]:
words_top_wiki = []
for common_word in cnt.most_common(1000):
    words_top_wiki.append(common_word[0])


#### t-SNE visualization

In [54]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [55]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)